In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings
import joblib
import os
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',None)

In [105]:
file_list = []
for root_dir,sub_dir,files in os.walk('./data'):
    file_list.append(files)
file_list = file_list[0]
dataframe_list = pd.DataFrame()
for file in files:
    if file.endswith('xls'):
        file_name = os.path.join(root_dir,file)
        df = pd.read_excel(file_name)
        dataframe_list = dataframe_list.append(df)

In [106]:
dataframe_list.drop(['Unnamed: 18','Unnamed: 19','Unnamed: 20','Unnamed: 21','Unnamed: 22','Unnamed: 23','创角账号数'],
                    axis=1,inplace=True)
dataframe_list.columns = ['plan_name','channel_id','source_id','data_win','source_run_datetime','create_time','plan_id',
                         'game_id','platform','source_run_date_amount','create_role_num','create_role_cost','create_role_pay_num',
                          'create_role_pay_cost','create_role_pay_sum','create_role_roi','create_role_retain_1d']
dataframe_list.dropna(how='any',inplace=True)

In [107]:
dataframe_list.shape

(1718817, 17)

In [108]:
dataframe_list = dataframe_list[dataframe_list['source_run_date_amount'] > 0]

In [110]:
dataframe_list.shape

(399792, 17)

In [111]:
dataframe_list.sort_values(by='data_win',inplace=True)

In [113]:
dataframe_list.drop_duplicates(subset=['channel_id','source_id','source_run_date_amount','create_role_num'],keep='first'
                               ,inplace=True)

In [114]:
dataframe_list.shape

(102843, 17)

In [116]:
dataframe_list['channel_id'] = dataframe_list['channel_id'].astype(np.int64)
dataframe_list['source_id'] = dataframe_list['source_id'].astype(np.int64)
dataframe_list['data_win'] = dataframe_list['data_win'].astype(np.int64)
dataframe_list['plan_id'] = dataframe_list['plan_id'].astype(np.int64)
dataframe_list['platform'] = dataframe_list['platform'].astype(np.int64)
dataframe_list['game_id'] = dataframe_list['game_id'].astype(np.int64)
dataframe_list['source_run_datetime'] = pd.to_datetime(dataframe_list['source_run_datetime'])

In [117]:
dataframe_list.head()

,plan_name,channel_id,source_id,data_win,source_run_datetime,create_time,plan_id,game_id,platform,source_run_date_amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_retain_1d
1,番茄小说-末日安卓,7162,299591,0,2020-05-01,2020-04-24,289813,1001049,1,200000.00,690.0,290.275762,3.0,66666.666667,318.0,0.00159,0.0522
197,HSX-TT-WQ-废土-0518_4,20541,311564,0,2020-05-19,2020-05-18,301786,1000993,2,657.55,3.0,219.183333,0.0,0.000000,0.0,0.00000,0.0000
198,HSX-TT-WQ-废土-0518_6,20541,311566,0,2020-05-19,2020-05-18,301788,1000993,2,384.88,3.0,128.293333,0.0,0.000000,0.0,0.00000,0.0000
199,mei-末日-new-5.18_2,20953,311781,0,2020-05-19,2020-05-18,302003,1001049,1,274.87,3.0,91.623333,0.0,0.000000,0.0,0.00000,0.0000
200,zhu-废土-IOS-5.18_2,20954,311786,0,2020-05-19,2020-05-18,302008,1000993,2,274.48,3.0,91.493333,0.0,0.000000,0.0,0.00000,0.0000


In [118]:
# 读取头条5-7月份计划30日回款信息
pay_info = pd.read_csv('./30_d_pay_sum.csv')
pay_info['source_run_datetime'] = pd.to_datetime(pay_info['source_run_datetime'])
pay_info.drop(pay_info[pay_info['channel_id']==-1].index,inplace=True)

In [232]:
pay_info.shape

(60118, 4)

In [218]:
data_pay = dataframe_list[dataframe_list['data_win'] == 0]

In [233]:
data_pay.shape

(20694, 5)

In [219]:
data_pay = pd.merge(data_pay,pay_info,on=['channel_id','source_id','source_run_datetime'],how='left',
                validate='one_to_one')
data_pay.dropna(how='any',inplace=True)

In [220]:
data_pay['30_ROI'] = round(data_pay['30_d_pay_sum']/data_pay['source_run_date_amount'],6)

In [224]:
data_pay = data_pay[['channel_id','source_id','source_run_datetime','30_d_pay_sum','30_ROI']]

In [225]:
data_pay.head()

,channel_id,source_id,source_run_datetime,30_d_pay_sum,30_ROI
0,7162,299591,2020-05-01,4654.0,0.02327
1,20541,311564,2020-05-19,0.0,0.00000
2,20541,311566,2020-05-19,0.0,0.00000
3,20953,311781,2020-05-19,0.0,0.00000
4,20954,311786,2020-05-19,0.0,0.00000


In [227]:
data = pd.merge(dataframe_list,data_pay,on=['channel_id','source_id','source_run_datetime'],how='left',
                validate='many_to_one')
data.dropna(how='any',inplace=True)
data.head()

,plan_name,channel_id,source_id,data_win,source_run_datetime,create_time,plan_id,game_id,platform,source_run_date_amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_retain_1d,30_d_pay_sum,30_ROI
0,番茄小说-末日安卓,7162,299591,0,2020-05-01,2020-04-24,289813,1001049,1,200000.00,690.0,290.275762,3.0,66666.666667,318.0,0.00159,0.0522,4654.0,0.02327
1,HSX-TT-WQ-废土-0518_4,20541,311564,0,2020-05-19,2020-05-18,301786,1000993,2,657.55,3.0,219.183333,0.0,0.000000,0.0,0.00000,0.0000,0.0,0.00000
2,HSX-TT-WQ-废土-0518_6,20541,311566,0,2020-05-19,2020-05-18,301788,1000993,2,384.88,3.0,128.293333,0.0,0.000000,0.0,0.00000,0.0000,0.0,0.00000
3,mei-末日-new-5.18_2,20953,311781,0,2020-05-19,2020-05-18,302003,1001049,1,274.87,3.0,91.623333,0.0,0.000000,0.0,0.00000,0.0000,0.0,0.00000
4,zhu-废土-IOS-5.18_2,20954,311786,0,2020-05-19,2020-05-18,302008,1000993,2,274.48,3.0,91.493333,0.0,0.000000,0.0,0.00000,0.0000,0.0,0.00000


In [228]:
data.shape

(44703, 19)

In [235]:
data['data_win'].value_counts()

0    20694
2     8156
3     5328
4     3752
5     2794
6     2164
7     1736
1       79
Name: data_win, dtype: int64

In [272]:
data_ignore = data[data['source_run_date_amount'] < 500]
data_not_ignore = data[data['source_run_date_amount'] >= 500]

In [273]:
data_an = data_not_ignore[data_not_ignore['platform'] == 1]
data_ios = data_not_ignore[data_not_ignore['platform'] == 2]

In [274]:
data_win_0 = data_an[data_an['data_win'] == 0]
data_win_1 = data_an[data_an['data_win'] == 1]
data_win_2 = data_an[data_an['data_win'] == 2]
data_win_3 = data_an[data_an['data_win'] == 3]
data_win_4 = data_an[data_an['data_win'] == 4]
data_win_5 = data_an[data_an['data_win'] == 5]
data_win_6 = data_an[data_an['data_win'] == 6]
data_win_7 = data_an[data_an['data_win'] == 7]
data_win_0_ios = data_ios[data_ios['data_win'] == 0]
data_win_1_ios = data_ios[data_ios['data_win'] == 1]
data_win_2_ios = data_ios[data_ios['data_win'] == 2]
data_win_3_ios = data_ios[data_ios['data_win'] == 3]
data_win_4_ios = data_ios[data_ios['data_win'] == 4]
data_win_5_ios = data_ios[data_ios['data_win'] == 5]
data_win_6_ios = data_ios[data_ios['data_win'] == 6]
data_win_7_ios = data_ios[data_ios['data_win'] == 7]

In [275]:
print(data_win_0[data_win_0['30_ROI'] > 0.4]['source_run_date_amount'].sum())
print(data_win_0[data_win_0['30_ROI'] > 0.4]['create_role_num'].sum())
print(data_win_0[data_win_0['30_ROI'] > 0.4]['source_run_date_amount'].sum()/data_win_0[data_win_0['30_ROI'] > 0.4]['create_role_num'].sum())

3769735.67
210354.0
17.920912699544576


In [276]:
print(data_win_0[data_win_0['30_ROI'] < 0.4]['source_run_date_amount'].sum())
print(data_win_0[data_win_0['30_ROI'] < 0.4]['create_role_num'].sum())
print(data_win_0[data_win_0['30_ROI'] < 0.4]['source_run_date_amount'].sum()/data_win_0[data_win_0['30_ROI'] < 0.4]['create_role_num'].sum())

17138689.020000003
732405.0
23.400562557601333


In [277]:
data_win_0['label'] = data_win_0.apply(lambda x:1 if x.create_role_num == 0 else 
                                (1 if x.create_role_cost>=30 else (1 if x.create_role_pay_sum==0 else 0) ),axis=1)

In [278]:
data_win_0['label'].value_counts()

1    7340
0    1258
Name: label, dtype: int64

In [279]:
print(data_win_0_ios[data_win_0_ios['30_ROI'] > 0.3]['source_run_date_amount'].sum())
print(data_win_0_ios[data_win_0_ios['30_ROI'] > 0.3]['create_role_num'].sum())
print(data_win_0_ios[data_win_0_ios['30_ROI'] > 0.3]['source_run_date_amount'].sum()/data_win_0_ios[data_win_0_ios['30_ROI'] > 0.3]['create_role_num'].sum())

1444682.88
32085.0
45.02673772791024


In [280]:
print(data_win_0_ios[data_win_0_ios['30_ROI'] < 0.3]['source_run_date_amount'].sum())
print(data_win_0_ios[data_win_0_ios['30_ROI'] < 0.3]['create_role_num'].sum())
print(data_win_0_ios[data_win_0_ios['30_ROI'] < 0.3]['source_run_date_amount'].sum()/data_win_0_ios[data_win_0_ios['30_ROI'] < 0.3]['create_role_num'].sum())

6926063.46
116584.0
59.40835329033143


In [283]:
data_win_0_ios['label'] = data_win_0_ios.apply(lambda x:1 if x.create_role_num == 0 else 
                                (1 if x.create_role_cost>=60 else (1 if x.create_role_pay_sum==0 else 0) ),axis=1)

In [284]:
data_win_0_ios['label'].value_counts()

1    3342
0     312
Name: label, dtype: int64

# win = 1

In [285]:
data_win_1['label'] = data_win_1.apply(lambda x:1 if x.create_role_num == 0 else 
                                (1 if x.create_role_cost>=30 else (1 if x.create_role_pay_sum==0 else 0) ),axis=1)
data_win_1_ios['label'] = data_win_1_ios.apply(lambda x:1 if x.create_role_num == 0 else 
                                (1 if x.create_role_cost>=60 else (1 if x.create_role_pay_sum==0 else 0) ),axis=1)

# win = 2

In [286]:
data_win_2.head()

,plan_name,channel_id,source_id,data_win,source_run_datetime,create_time,plan_id,game_id,platform,source_run_date_amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_retain_1d,30_d_pay_sum,30_ROI
25899,HSX-TT-YX-末日-0527_1,20540,315443,2,2020-05-29,2020-05-27,305665,1000840,1,2384.15,16.0,149.009375,1.0,2384.15,68.0,0.028522,0.0625,0.0,0.000000
25902,Li-fpftt-mrwc_5.27_12,21116,315495,2,2020-05-29,2020-05-27,305717,1000840,1,1400.60,17.0,82.388235,1.0,1400.60,72.0,0.051407,0.0588,0.0,0.000000
25915,ku11-mrwc-7.17_19,20754,332076,2,2020-07-21,2020-07-17,322296,1000840,1,5794.75,428.0,13.570843,5.0,1158.95,156.0,0.026921,0.0820,24.0,0.008463
25916,HSX-TT-CH104-末日危城-SSR1417_4,20540,332602,2,2020-07-21,2020-07-18,322822,1000840,1,5844.23,426.0,13.751129,0.0,0.00,0.0,0.000000,0.1059,236.0,0.139854
25918,Fu-fpftt-mrwc_7.16_85,21116,331495,2,2020-07-21,2020-07-16,321715,1000840,1,5745.22,391.0,15.039843,2.0,2872.61,24.0,0.004177,0.0681,336.0,0.137300


In [287]:
print(data_win_2[data_win_2['30_ROI'] > 0.4]['source_run_date_amount'].sum())
print(data_win_2[data_win_2['30_ROI'] > 0.4]['create_role_pay_sum'].sum())
print(data_win_2[data_win_2['30_ROI'] > 0.4]['create_role_pay_sum'].sum()/data_win_2[data_win_2['30_ROI'] > 0.4]['source_run_date_amount'].sum())

3413676.87
231186.0
0.06772345737574159


In [288]:
print(data_win_2[data_win_2['30_ROI'] < 0.4]['source_run_date_amount'].sum())
print(data_win_2[data_win_2['30_ROI'] < 0.4]['create_role_pay_sum'].sum())
print(data_win_2[data_win_2['30_ROI'] < 0.4]['create_role_pay_sum'].sum()/data_win_2[data_win_2['30_ROI'] < 0.4]['source_run_date_amount'].sum())

18060500.75
857556.0
0.04748240438460711


In [289]:
print(data_win_2_ios[data_win_2_ios['30_ROI'] > 0.3]['source_run_date_amount'].sum())
print(data_win_2_ios[data_win_2_ios['30_ROI'] > 0.3]['create_role_pay_sum'].sum())
print(data_win_2_ios[data_win_2_ios['30_ROI'] > 0.3]['create_role_pay_sum'].sum()/data_win_2_ios[data_win_2_ios['30_ROI'] > 0.3]['source_run_date_amount'].sum())

1527353.27
69388.0
0.0454302232253053


In [290]:
print(data_win_2_ios[data_win_2_ios['30_ROI'] < 0.3]['source_run_date_amount'].sum())
print(data_win_2_ios[data_win_2_ios['30_ROI'] < 0.3]['create_role_pay_sum'].sum())
print(data_win_2_ios[data_win_2_ios['30_ROI'] < 0.3]['create_role_pay_sum'].sum()/data_win_2_ios[data_win_2_ios['30_ROI'] < 0.3]['source_run_date_amount'].sum())

6802233.5600000005
325368.0
0.04783252399819097


In [291]:
data_win_2['label'] = data_win_2.apply(lambda x:1 if x.create_role_num == 0 else 
                                (1 if x.create_role_cost>=30 else (1 if x.create_role_roi<=0.04 else 0) ),axis=1)
data_win_2_ios['label'] = data_win_2_ios.apply(lambda x:1 if x.create_role_num == 0 else 
                                (1 if x.create_role_cost>=60 else (1 if x.create_role_roi<=0.03 else 0) ),axis=1)

In [292]:
data_win_2['label'].value_counts()

1    4591
0     499
Name: label, dtype: int64

In [293]:
data_win_2_ios['label'].value_counts()

1    1929
0     184
Name: label, dtype: int64

# win = 3

In [294]:
data_win_3.shape

(3471, 19)

In [295]:
data_win_3.head()

,plan_name,channel_id,source_id,data_win,source_run_datetime,create_time,plan_id,game_id,platform,source_run_date_amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_retain_1d,30_d_pay_sum,30_ROI
46496,ZO-fpftt-mrwc_6.6_70,21116,317271,3,2020-06-09,2020-06-06,307493,1000840,1,2095.98,59.0,35.525085,0.0,0.0,0.0,0.0,0.0847,0.0,0.000000
46512,cxq-ttpan生机0605_6,20646,317227,3,2020-06-09,2020-06-05,307449,1001049,1,1730.19,47.0,36.812553,0.0,0.0,0.0,0.0,0.0426,0.0,0.000000
46515,cxq-ttlai危城0606_5,20646,317346,3,2020-06-09,2020-06-06,307568,1000840,1,3312.91,48.0,69.018958,0.0,0.0,0.0,0.0,0.0833,0.0,0.000000
46521,ROI- Wu-ftt-末日_7.25_177,21116,335741,3,2020-07-31,2020-07-25,325961,1000840,1,1138.44,19.0,59.917895,0.0,0.0,0.0,0.0,0.0000,0.0,0.000000
46532,7ttyrf_末日ROI_7.28_5,6770,336715,3,2020-07-31,2020-07-28,326932,1000840,1,2958.51,39.0,77.855526,0.0,0.0,0.0,0.0,0.0526,238.0,0.389927


In [296]:
data_win_3[data_win_3['create_role_pay_num']==0].shape

(1254, 19)

In [297]:
print(data_win_3[data_win_3['30_ROI'] > 0.4]['source_run_date_amount'].sum())
print(data_win_3[data_win_3['30_ROI'] > 0.4]['create_role_pay_num'].sum())
print(data_win_3[data_win_3['30_ROI'] > 0.4]['source_run_date_amount'].sum()/data_win_3[data_win_3['30_ROI'] > 0.4]['create_role_pay_num'].sum())

3589686.24
1477.0
2430.390142180095


In [298]:
print(data_win_3[data_win_3['30_ROI'] < 0.4]['source_run_date_amount'].sum())
print(data_win_3[data_win_3['30_ROI'] < 0.4]['create_role_pay_num'].sum())
print(data_win_3[data_win_3['30_ROI'] < 0.4]['source_run_date_amount'].sum()/data_win_3[data_win_3['30_ROI'] < 0.4]['create_role_pay_num'].sum())

19596677.87
7071.0
2771.4153401216236


In [299]:
print(data_win_3_ios[data_win_3_ios['30_ROI'] > 0.3]['source_run_date_amount'].sum())
print(data_win_3_ios[data_win_3_ios['30_ROI'] > 0.3]['create_role_pay_num'].sum())
print(data_win_3_ios[data_win_3_ios['30_ROI'] > 0.3]['source_run_date_amount'].sum()/data_win_3_ios[data_win_3_ios['30_ROI'] > 0.3]['create_role_pay_num'].sum())

1565094.4700000002
394.0
3972.320989847716


In [300]:
print(data_win_3_ios[data_win_3_ios['30_ROI'] < 0.3]['source_run_date_amount'].sum())
print(data_win_3_ios[data_win_3_ios['30_ROI'] < 0.3]['create_role_pay_num'].sum())
print(data_win_3_ios[data_win_3_ios['30_ROI'] < 0.3]['source_run_date_amount'].sum()/data_win_3_ios[data_win_3_ios['30_ROI'] < 0.3]['create_role_pay_num'].sum())

7029432.300000001
1631.0
4309.891048436542


In [458]:
print(data_win_3[data_win_3['30_ROI'] > 0.4]['source_run_date_amount'].sum())
print(data_win_3[data_win_3['30_ROI'] > 0.4]['create_role_pay_sum'].sum())
print(data_win_3[data_win_3['30_ROI'] > 0.4]['create_role_pay_sum'].sum()/data_win_3[data_win_3['30_ROI'] > 0.4]['source_run_date_amount'].sum())

3589686.24
309134.0
0.08611727580959833


In [459]:
print(data_win_3[data_win_3['30_ROI'] < 0.4]['source_run_date_amount'].sum())
print(data_win_3[data_win_3['30_ROI'] < 0.4]['create_role_pay_sum'].sum())
print(data_win_3[data_win_3['30_ROI'] < 0.4]['create_role_pay_sum'].sum()/data_win_3[data_win_3['30_ROI'] < 0.4]['source_run_date_amount'].sum())

19596677.87
1507582.0
0.07693048842262773


In [460]:
print(data_win_3_ios[data_win_3_ios['30_ROI'] > 0.3]['source_run_date_amount'].sum())
print(data_win_3_ios[data_win_3_ios['30_ROI'] > 0.3]['create_role_pay_sum'].sum())
print(data_win_3_ios[data_win_3_ios['30_ROI'] > 0.3]['create_role_pay_sum'].sum()/data_win_3_ios[data_win_3_ios['30_ROI'] > 0.3]['source_run_date_amount'].sum())

1565094.4700000002
80058.0
0.051152183803959125


In [461]:
print(data_win_3_ios[data_win_3_ios['30_ROI'] < 0.3]['source_run_date_amount'].sum())
print(data_win_3_ios[data_win_3_ios['30_ROI'] < 0.3]['create_role_pay_sum'].sum())
print(data_win_3_ios[data_win_3_ios['30_ROI'] < 0.3]['create_role_pay_sum'].sum()/data_win_3_ios[data_win_3_ios['30_ROI'] < 0.3]['source_run_date_amount'].sum())

7029432.300000001
568392.0
0.08085887675452823


In [454]:
print(data_win_3[data_win_3['30_ROI'] < 0.4]['create_role_retain_1d'].mean())
print(data_win_3[data_win_3['30_ROI'] > 0.4]['create_role_retain_1d'].mean())

0.09837724358974348
0.08766866096866094


In [455]:
print(data_win_3_ios[data_win_3_ios['30_ROI'] < 0.3]['create_role_retain_1d'].mean())
print(data_win_3_ios[data_win_3_ios['30_ROI'] > 0.3]['create_role_retain_1d'].mean())

0.10021851278079016
0.09067941176470595


In [495]:
data_win_3['label'] = data_win_3.apply(lambda x:1 if x.create_role_num == 0 else(1 if x.create_role_pay_num==0 else (1 if x.create_role_pay_cost > 3000 else (1 if x.create_role_retain_1d<0.07 else(1 if x.create_role_roi<0.08 else 0)))),axis=1)
data_win_3_ios['label'] = data_win_3_ios.apply(lambda x:1 if x.create_role_num == 0 else(1 if x.create_role_pay_num==0 else (1 if x.create_role_pay_cost >6000 else (1 if x.create_role_retain_1d<0.07 else(1 if x.create_role_roi<0.06 else 0)))),axis=1)

In [496]:
data_win_3['label'].value_counts()

1    3203
0     268
Name: label, dtype: int64

In [497]:
data_win_3_ios['label'].value_counts()

1    1301
0     126
Name: label, dtype: int64

# win = 7

In [501]:
data_win_6['label'] = data_win_6.apply(lambda x:1 if x.create_role_num == 0 else(1 if x.create_role_pay_num==0 else (1 if x.create_role_cost>=150 else (1 if x.create_role_pay_cost >6000 else (0 if x.create_role_roi>=0.13 else 1)))),axis=1)
data_win_6_ios['label'] = data_win_6_ios.apply(lambda x:1 if x.create_role_num == 0 else(1 if x.create_role_pay_num==0 else (1 if x.create_role_cost>=300 else (1 if x.create_role_pay_cost >10000 else (0 if x.create_role_roi>=0.12 else 1)))),axis=1)

In [502]:
data_win_6['label'].value_counts()

1    1049
0     398
Name: label, dtype: int64

In [503]:
data_win_6_ios['label'].value_counts()

1    429
0    171
Name: label, dtype: int64